# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load to cities csv 
file_to_load ="output_data/cities.csv"

city_df= pd.read_csv(file_to_load)
city_df.head()

,City_ID,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,0,2246901,Oussouye,SN,12.4850,-16.5469,68.65,52,0,7.76
1,1,5122534,Jamestown,US,42.0970,-79.2353,32.38,34,40,8.10
2,2,3981069,Tres Palos,MX,18.1833,-101.6167,66.92,95,0,3.44
3,3,2022572,Khatanga,RU,71.9667,102.5000,-30.12,94,69,8.23
4,4,4513583,Hamilton,US,39.1834,-84.5333,36.54,87,90,6.91


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#lesson plan/6/3/7
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure(center=(0,0),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 60

heat_layer.point_radius = 3



fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
city_df.describe()

,City_ID,City ID,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
count,554.000000,5.540000e+02,554.000000,554.000000,554.00000,554.000000,554.000000,554.000000
mean,276.500000,2.825265e+06,20.942106,15.247181,52.02074,72.812274,55.767148,7.592166
std,160.070297,1.703412e+06,33.730606,91.735713,26.96713,21.667989,38.808590,5.450946
min,0.000000,5.081400e+04,-54.800000,-175.200000,-30.12000,10.000000,0.000000,0.540000
25%,138.250000,1.652747e+06,-7.324050,-68.291675,32.00000,62.000000,15.500000,3.440000
50%,276.500000,2.367478e+06,27.996950,17.596500,59.77500,79.000000,72.500000,5.990000
75%,414.750000,3.859595e+06,48.510900,97.693200,74.03500,89.000000,90.000000,10.360000
max,553.000000,1.072286e+07,78.218600,179.316700,96.80000,100.000000,100.000000,31.070000


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#sorted 
sorted_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 85)] 
sorted_df = sorted_df[sorted_df['Wind Speed'] <= 5]
sorted_df = sorted_df[sorted_df['Clouds'] <= 10]

sorted_df = sorted_df[sorted_df['Humidity'] <= 65]
hotel_df = sorted_df
hotel_df.head()




,City_ID,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
67,67,1295765,Sittwe,MM,20.1500,92.9000,83.44,34,0,1.61
106,106,286621,Salalah,OM,17.0151,54.0924,75.20,38,0,1.14
113,113,1278969,Along,IN,28.1667,94.7667,81.37,46,0,0.72
160,160,286245,Sur,OM,22.5667,59.5289,77.58,65,0,4.90
218,218,1809061,Jiaojiang,CN,28.6803,121.4428,78.01,61,0,3.00


In [7]:
#hotel_df.describe()
#hotel_df.count()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create new column in hotel_df
hotel_df["Hotel Name"]=""
hotel_df



,City_ID,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
67,67,1295765,Sittwe,MM,20.1500,92.9000,83.44,34,0,1.61,
106,106,286621,Salalah,OM,17.0151,54.0924,75.20,38,0,1.14,
113,113,1278969,Along,IN,28.1667,94.7667,81.37,46,0,0.72,
160,160,286245,Sur,OM,22.5667,59.5289,77.58,65,0,4.90,
218,218,1809061,Jiaojiang,CN,28.6803,121.4428,78.01,61,0,3.00,
351,351,1282898,Pokhara,NP,28.2333,83.9833,80.26,23,0,4.99,
413,413,1609070,Loei,TH,17.4167,101.5000,84.20,65,3,2.30,


In [9]:
#LESSONPLAN/6/ACTIVITIES 4/3

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"keyword": "hotel", "radius": 5000, "key": g_key,}

for index, row in hotel_df.iterrows():
    try:
        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NA"
        
hotel_df



,City_ID,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
67,67,1295765,Sittwe,MM,20.1500,92.9000,83.44,34,0,1.61,Hotel Memory
106,106,286621,Salalah,OM,17.0151,54.0924,75.20,38,0,1.14,Salalah Gardens Hotel
113,113,1278969,Along,IN,28.1667,94.7667,81.37,46,0,0.72,Hotel West
160,160,286245,Sur,OM,22.5667,59.5289,77.58,65,0,4.90,Sur Plaza Hotel
218,218,1809061,Jiaojiang,CN,28.6803,121.4428,78.01,61,0,3.00,Hilton Taizhou
351,351,1282898,Pokhara,NP,28.2333,83.9833,80.26,23,0,4.99,Shangrila Village Resort Pokhara
413,413,1609070,Loei,TH,17.4167,101.5000,84.20,65,3,2.30,Pruksatara Garden


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map(lesson plan-API-3-6 )

markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))